## Predict Future Sales

These notebook was based on three kernels, mostly on the first one. Thank very much [**dlarionov**](https://www.kaggle.com/dlarionov/), [**dimitreoliveira**](https://www.kaggle.com/dimitreoliveira/), and [**kyakovlev**](https://www.kaggle.com/kyakovlev/)!

- https://www.kaggle.com/dlarionov/feature-engineering-xgboost
- https://www.kaggle.com/dimitreoliveira/model-stacking-feature-engineering-and-eda
- https://www.kaggle.com/kyakovlev/1st-place-solution-part-1-hands-on-data

### 1) Exploratory Data Analysis

#### Importing Packages and Datasets

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import itertools
from sklearn.preprocessing import LabelEncoder


sns.set()

train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
train['date'] = pd.to_datetime(train['date'], format = '%d.%m.%Y', infer_datetime_format = True)

cats = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
sub = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv').set_index('ID')

#### Outliers

In [ ]:
f, axes = plt.subplots(2, 2, figsize = (17,9))
g0 = sns.distplot(train.item_price, kde = False, ax = axes[0,0])
g1 = sns.boxplot(train.item_price, ax = axes[0,1])
g2 = sns.distplot(train.item_price[train.item_price.between(0,10000)], bins = 100, kde = False, ax = axes[1,0])
g3 = sns.boxplot(train.item_price[train.item_price.between(0,10000)], ax = axes[1,1], fliersize = 2)

f.suptitle('Item Price - Distribution and Boxplot', fontsize = 16, fontweight='bold')
g0.set_xlabel('Item Price', size = 14)
g1.set_xlabel('Item Price', size = 14)
g2.set_xlabel('Item Price', size = 14)
g3.set_xlabel('Item Price', size = 14);

In [ ]:
f, axes = plt.subplots(2, 2, figsize = (17,9))
g0 = sns.distplot(train.item_cnt_day, kde = False, ax = axes[0,0])
g1 = sns.boxplot(train.item_cnt_day, ax = axes[0,1])
g2 = sns.distplot(train.item_cnt_day[train.item_cnt_day.between(0,100)], bins = 100, kde = False, ax = axes[1,0])
g3 = sns.boxplot(train.item_cnt_day[train.item_cnt_day.between(0,100)], ax = axes[1,1], fliersize = 2)

f.suptitle('Item Count per Day - Distribution and Boxplot', fontsize=16, fontweight='bold')
g0.set_xlabel('Item Count per Day', size = 14)
g1.set_xlabel('Item Count per Day', size = 14)
g2.set_xlabel('Item Count per Day', size = 14)
g3.set_xlabel('Item Count per Day', size = 14);

#### Sold Items x Shops

In [ ]:
f, axes = plt.subplots(1, 1, figsize = (17,5))
g = sns.barplot(x = 'shop_id', y = 'item_cnt_day', data = train,
                estimator = lambda x: np.sum(x) / np.sum(train.item_cnt_day) * 100,
                order = train.groupby('shop_id').sum()['item_cnt_day'].sort_values(ascending = False).index,
                color = 'orange', ci = None)
g.set_xlabel('Shop ID', size = 14)
g.set_ylabel('Sold Items (%)', size = 14);

#### Time Series Visualization

In [ ]:
train_month_year = train.groupby(['date_block_num']).sum().reset_index().drop(columns = ['item_price', 'shop_id', 'item_id'])

f, axes = plt.subplots(1, 1, figsize = (17,5))
g = sns.lineplot(x = 'date_block_num', y = 'item_cnt_day', data = train_month_year)
g.set_xlabel('Month', size = 14)
g.set_ylabel('Item Count Month', size = 14)
g.set_xlim(0, 33);

#### Outdated items

In [ ]:
sales_by_item_id = train.pivot_table(index = ['item_id'], values = ['item_cnt_day'], columns = 'date_block_num',
                                     aggfunc = np.sum, fill_value = 0).reset_index()
sales_by_item_id.columns = ['item_id'] + list(sales_by_item_id.columns.droplevel()[1:].map(str))

outdated_items_3mo = sales_by_item_id[sales_by_item_id.loc[:,'31':].sum(axis=1)==0]
outdated_items_6mo = sales_by_item_id[sales_by_item_id.loc[:,'28':].sum(axis=1)==0]

print('Outdated items (3 months):', 100.0*outdated_items_3mo.shape[0]/sales_by_item_id.shape[0])
print('Outdated items (6 months):', 100.0*outdated_items_6mo.shape[0]/sales_by_item_id.shape[0])

#### Outdated shops

In [ ]:
sales_by_shop_id = train.pivot_table(index = ['shop_id'], values = ['item_cnt_day'], columns = 'date_block_num',
                                     aggfunc = np.sum, fill_value = 0).reset_index()
sales_by_shop_id.columns = ['shop_id'] + list(sales_by_shop_id.columns.droplevel()[1:].map(str))

outdated_shop_3mo = sales_by_shop_id[sales_by_shop_id.loc[:,'31':].sum(axis=1)==0]
outdated_shop_6mo = sales_by_shop_id[sales_by_shop_id.loc[:,'28':].sum(axis=1)==0]

print('Outdated shops (3 months):', 100.0*outdated_shop_3mo.shape[0]/sales_by_shop_id.shape[0])
print('Outdated shops (6 months):', 100.0*outdated_shop_6mo.shape[0]/sales_by_shop_id.shape[0])

### 2) Feature Engineering

#### Organizing and Extracting Features - Shops

In [ ]:
train_featured = train.copy()
test_featured = test.copy()

train_featured.loc[train_featured.shop_id == 11, 'shop_id'] = 10
test_featured.loc[test_featured.shop_id == 11, 'shop_id'] = 10

train_featured.loc[train_featured.shop_id == 1, 'shop_id'] = 58
test_featured.loc[test_featured.shop_id == 1, 'shop_id'] = 58

train_featured.loc[train_featured.shop_id == 0, 'shop_id'] = 57
test_featured.loc[test_featured.shop_id == 0, 'shop_id'] = 57

train_featured.loc[train_featured.shop_id == 40, 'shop_id'] = 39
test_featured.loc[test_featured.shop_id == 40, 'shop_id'] = 39

shops_featured = shops.copy()
shops_featured['shop_name'] = shops_featured['shop_name'].apply(lambda x: x.lower()).str.replace('[^\w\s]', '').str.replace('\d+','').str.strip()
shops_featured['shop_city'] = shops_featured['shop_name'].str.partition(' ')[0]
shops_featured['shop_type'] = shops_featured['shop_name'].apply(lambda x: 'мтрц' if 'мтрц' in x else 'трц' if 'трц' in x else 'трк' if 'трк' in x else 'тц' if 'тц' in x else 'тк' if 'тк' in x else 'NO_DATA')

shops_featured['shop_city'] = LabelEncoder().fit_transform(shops_featured['shop_city'])
shops_featured['shop_type'] = LabelEncoder().fit_transform(shops_featured['shop_type'])

shops_featured.drop(columns = 'shop_name', inplace = True)
shops_featured.head()

#### Organizing and Extracting Features - Categories

In [ ]:
cats_featured = cats.copy()
cats_featured['split'] = cats_featured['item_category_name'].str.split('-')
cats_featured['category_type'] = cats_featured['split'].map(lambda x: x[0].strip())
cats_featured['category_subtype'] = cats_featured['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())

cats_featured['category_type'] = LabelEncoder().fit_transform(cats_featured['category_type'])
cats_featured['category_subtype'] = LabelEncoder().fit_transform(cats_featured['category_subtype'])


cats_featured = cats_featured[['item_category_id','category_type', 'category_subtype']]
cats_featured.head()

#### Organizing and Extracting Features - Items

In [ ]:
items_featured = items.copy()
items_featured.drop(columns = ['item_name'], inplace = True)
items_featured.head()

#### Organizing dataset

In [ ]:
# # Creating base dataframe with all the possible permutations between shops and items present on the test set
# import itertools

# ts = time.time()

# date_block_nums = np.array(range(35))
# shops_test = test.shop_id.unique()
# items_test = test.item_id.unique()
# base = pd.DataFrame(itertools.product(date_block_nums, shops_test, items_test),
#                     columns = ['date_block_num','shop_id','item_id'])

# time.time() - ts

In [ ]:
from itertools import product

ts = time.time()

base = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = train_featured[train_featured.date_block_num==i]
    base.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype = 'int16'))
    
base = pd.DataFrame(np.vstack(base), columns=cols)
base.sort_values(cols,inplace = True)

time.time() - ts

In [ ]:
#Creating Revenue Feature
train_featured['revenue'] = train_featured['item_price'] *  train_featured['item_cnt_day']

# Eliminating "item_cnt_day" and "item_price" outliers
train_featured = train_featured[(train_featured.item_cnt_day.between(0,1000)) & (train_featured.item_price.between(0,100000))]

# Aggregating by month
train_featured_month = (train_featured[['date_block_num', 'shop_id', 'item_id', 'item_cnt_day']]
                        .groupby(['date_block_num','shop_id','item_id'])
                        .sum())
train_featured_month.reset_index(inplace = True)
train_featured_month['item_cnt_month'] = train_featured_month['item_cnt_day'].clip(0,20).fillna(0)
train_featured_month.drop(columns = 'item_cnt_day', inplace = True)

# Preparating test set
test_featured['date_block_num'] = 34
test_featured = test_featured[['date_block_num', 'shop_id', 'item_id']]

# Concatenating train and test set
base = pd.concat([base, test_featured])

# Concatenating shops and categories information
df_raw = pd.merge(base, train_featured_month, on = ['date_block_num', 'shop_id', 'item_id'], how = 'left').fillna(0)
df_raw = pd.merge(df_raw, shops_featured, on = ['shop_id'], how='left')
df_raw = pd.merge(df_raw, items_featured, on = ['item_id'], how='left')
df_raw = pd.merge(df_raw, cats_featured, on = ['item_category_id'], how='left')

# Adding month and year feature
df_raw['year'] = df_raw['date_block_num'].apply(lambda x: ((x//12) + 2013))
df_raw['month'] = df_raw['date_block_num'].apply(lambda x: (x % 12) + 1)

df_raw.head()

#### Lag Features

In [ ]:
df_raw_1 = df_raw.copy()

def lag_feature(df, lags, col):
    tmp = df[['date_block_num', 'shop_id', 'item_id', col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num', 'shop_id', 'item_id', col + '_lag_' + str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on = ['date_block_num', 'shop_id', 'item_id'], how = 'left')
    return df

df_raw_1 = lag_feature(df_raw_1, [1, 2, 3, 4, 5, 6], 'item_cnt_month')

dataset = df_raw_1.copy()

#### Trend Features

In [ ]:
ts = time.time()

# Add total average price per item feature
group = train_featured.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace = True)
dataset = pd.merge(dataset, group, on = ['item_id'], how = 'left')

# Add monthly average price per item feature
group = train_featured.groupby(['date_block_num', 'item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace = True)
dataset = pd.merge(dataset, group, on = ['date_block_num', 'item_id'], how = 'left')

lags = [1, 2, 3, 4, 5, 6]

def lag_feature_2(df, lags, col):
    tmp = df[['item_avg_item_price','date_block_num', 'shop_id', 'item_id', col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['item_avg_item_price', 'date_block_num', 'shop_id', 'item_id', col + '_lag_' + str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on = ['item_avg_item_price', 'date_block_num', 'shop_id', 'item_id'], how = 'left')
    return df

dataset = lag_feature_2(dataset, lags, 'date_item_avg_item_price')

for i in lags:
    dataset['delta_price_lag_' + str(i)] = \
        (dataset['date_item_avg_item_price_lag_' + str(i)] - dataset['item_avg_item_price']) / dataset['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_' + str(i)]:
            return row['delta_price_lag_' + str(i)]
    return 0
    
dataset['delta_price_lag'] = dataset.apply(select_trend, axis=1)
dataset['delta_price_lag'] = dataset['delta_price_lag'].astype(np.float16)
dataset['delta_price_lag'].fillna(0, inplace = True)

features_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    features_to_drop += ['date_item_avg_item_price_lag_' + str(i)]
    features_to_drop += ['delta_price_lag_' + str(i)]

dataset.drop(features_to_drop, axis = 1, inplace = True)

time.time() - ts

#### Months since the last sale for each shop/item pair and for item

In [ ]:
# ts = time.time()
# cache = {}
# dataset['item_shop_last_sale'] = -1
# dataset['item_shop_last_sale'] = dataset['item_shop_last_sale'].astype(np.int8)
# for idx, row in dataset.iterrows():    
#     key = str(row.item_id)+' '+str(row.shop_id)
#     if key not in cache:
#         if row.item_cnt_month!=0:
#             cache[key] = row.date_block_num
#     else:
#         last_date_block_num = cache[key]
#         dataset.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
#         cache[key] = row.date_block_num         

# ####################################

# cache = {}
# dataset['item_last_sale'] = -1
# dataset['item_last_sale'] = dataset['item_last_sale'].astype(np.int8)
# for idx, row in dataset.iterrows():    
#     key = row.item_id
#     if key not in cache:
#         if row.item_cnt_month!=0:
#             cache[key] = row.date_block_num
#     else:
#         last_date_block_num = cache[key]
#         if row.date_block_num>last_date_block_num:
#             dataset.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
#             cache[key] = row.date_block_num

# ####################################

# dataset['item_shop_first_sale'] = dataset['date_block_num'] - dataset.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
# dataset['item_first_sale'] = dataset['date_block_num'] - dataset.groupby('item_id')['date_block_num'].transform('min')
# time.time() - ts

### 3) Train/Test Split

- **`Train set`**: months 0 - 32

- **`Validation set`**: month 33

- **`Test set`**: month 34

#### Mean Encoding

In [ ]:
# features_to_encode = ['item_id',
#                       'shop_id', 'shop_city', 'shop_type',
#                       'item_category_id', 'category_type', 'category_subtype',
#                       'year', 'month']

# for feature_i in features_to_encode:
#     gp_feature_mean = dataset[dataset.date_block_num < 34].groupby([feature_i]).agg({'item_cnt_month': ['mean']})
#     gp_feature_mean.columns = [feature_i + '_mean']
#     gp_feature_mean.reset_index(inplace = True)

#     dataset = pd.merge(dataset, gp_feature_mean, on = [feature_i], how = 'left')

In [ ]:
list_features_to_encode = [['date_block_num'],
                           ['date_block_num', 'item_id'],
                           ['date_block_num', 'shop_id'],
                           ['date_block_num', 'item_category_id'],
                           ['date_block_num', 'shop_id', 'item_category_id']]

for list_feature_i in list_features_to_encode:
    print(list_feature_i)
    gp_feature_mean = dataset.groupby(list_feature_i).agg({'item_cnt_month': ['mean']})
    column_name = '_'.join(list_feature_i) + '_mean'
    gp_feature_mean.columns = [column_name]
    gp_feature_mean.reset_index(inplace = True)

    dataset = pd.merge(dataset, gp_feature_mean, on = list_feature_i, how = 'left')
    dataset = lag_feature(dataset, [1,2], column_name)
    dataset.drop([column_name], axis = 1, inplace=True)

#### Filling Missing Values

In [ ]:
for shop_id in dataset['shop_id'].unique():
    for column in dataset.columns:
        shop_median = dataset[(dataset['shop_id'] == shop_id)][column].median()
        
        dataset.loc[(dataset[column].isnull()) & (dataset['shop_id'] == shop_id), column] = shop_median

#### Saving Dataset

In [ ]:
del train
del cats
del items
del sub
del shops
del test
del train_featured
del cats_featured
del items_featured
del shops_featured
del test_featured
del df_raw
del df_raw_1
del base

In [ ]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

dataset = downcast_dtypes(dataset)

In [ ]:
dataset.to_pickle('final_dataset.pkl')

### 4) Model Training

- **XGBoost**

- **CatBoost**

- **LightGBM**

- **Model Stacking**

In [ ]:
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
import catboost
import pickle
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
dataset = pd.read_pickle('/kaggle/working/final_dataset.pkl')

train_set = dataset[dataset.date_block_num.between(12,32)]
validation_set = dataset[dataset.date_block_num == 33]
test_set = dataset[dataset.date_block_num == 34]

X_train = train_set.drop(columns = ['item_cnt_month'])
Y_train = train_set['item_cnt_month']

X_validation = validation_set.drop(columns = ['item_cnt_month'])
Y_validation = validation_set['item_cnt_month']

X_test = test_set.drop(columns = ['item_cnt_month'])
Y_test = test_set['item_cnt_month']

In [ ]:
del dataset
del train_set
del validation_set
del test_set

### 4.1) XGBoost

In [ ]:
dtrain = xgb.DMatrix(X_train, label = Y_train)
dvalidation = xgb.DMatrix(X_validation, label = Y_validation)
dtest = xgb.DMatrix(X_test, label = Y_test)

watchlist = [(dtrain, 'train'), (dvalidation, 'validation')]

params = {'objective': 'reg:squarederror',
          'tree_method': 'gpu_hist',
          'eval_metric': 'rmse',
          'eta': 0.1,
          'max_depth': 4,
          'random_state': 0}

model_xgb = xgb.train(params, dtrain, 1000, watchlist, early_stopping_rounds = 10)
pickle.dump(model_xgb, open("model_xgb.pickle.dat", "wb"))

### 4.2) Catboost

In [ ]:
model_cat = CatBoostRegressor(
    task_type = 'GPU',
    iterations = 1000,
    random_seed = 0,
    learning_rate = 0.1,
    od_type = 'Iter',
    od_wait = 10,
    eval_metric = 'RMSE'
)

model_cat.fit(
    X_train, Y_train,
    eval_set = (X_validation, Y_validation),
    logging_level = 'Verbose',
    plot = False
)

pickle.dump(model_cat, open("model_cat.pickle.dat", "wb"))

### 4.3) LightGBM

In [ ]:
params = {
    "objective" : "regression",
    #"device_type": "gpu",
    "metric" : "rmse",
    "learning_rate": 0.01,
    "seed": 0
}

lgtrain = lgb.Dataset(X_train, label = Y_train)
lgval = lgb.Dataset(X_validation, label = Y_validation)
lgtest = lgb.Dataset(X_test, label = Y_test)
evals_result = {}
model_light = lgb.train(params, lgtrain, 1000, 
                  valid_sets = [lgtrain, lgval], 
                  early_stopping_rounds = 20, 
                  evals_result = evals_result)

pickle.dump(model_light, open("model_light.pickle.dat", "wb"))

### 4.4) Model Stacking and Predictions

In [ ]:
# Importing Predictions
model_xgb = pickle.load(open("/kaggle/working/model_xgb.pickle.dat", "rb"))
model_cat = pickle.load(open("/kaggle/working/model_cat.pickle.dat", "rb"))
model_light = pickle.load(open("/kaggle/working/model_light.pickle.dat", "rb"))

# XGBoost Predictions
y_pred_train_xgb = model_xgb.predict(dtrain).clip(0,20)
y_pred_validation_xgb = model_xgb.predict(dvalidation).clip(0,20)
y_pred_test_xgb = model_xgb.predict(dtest).clip(0,20)

# Catboost Predictions
y_pred_train_cat = model_cat.predict(X_train).clip(0,20)
y_pred_validation_cat = model_cat.predict(X_validation).clip(0,20)
y_pred_test_cat = model_cat.predict(X_test).clip(0,20)

# LightGBM Predictions
y_pred_train_light = model_light.predict(X_train).clip(0,20)
y_pred_validation_light = model_light.predict(X_validation).clip(0,20)
y_pred_test_light = model_light.predict(X_test).clip(0,20)

# Mounting train dataframe for stacking
df_train = pd.concat([pd.DataFrame(y_pred_train_xgb), pd.DataFrame(y_pred_train_cat), pd.DataFrame(y_pred_train_light), Y_train.reset_index()['item_cnt_month']], axis = 1)
df_train.columns = ['XGBoost', 'Catboost', 'LightGBM', 'Y']

# Mounting vaidation dataframe for stacking
df_validation = pd.concat([pd.DataFrame(y_pred_validation_xgb), pd.DataFrame(y_pred_validation_cat), pd.DataFrame(y_pred_validation_light), Y_validation.reset_index()['item_cnt_month']], axis = 1)
df_validation.columns = ['XGBoost', 'Catboost', 'LightGBM', 'Y']

# Mounting test dataframe for stacking
df_test = pd.concat([pd.DataFrame(y_pred_test_xgb), pd.DataFrame(y_pred_test_cat), pd.DataFrame(y_pred_test_light), Y_test.reset_index()['item_cnt_month']], axis = 1)
df_test.columns = ['XGBoost', 'Catboost', 'LightGBM', 'Y']

In [ ]:
meta_model = LinearRegression(n_jobs = -1)

df_train_X = df_train.drop(columns = 'Y')
df_train_Y = df_train['Y']

df_validation_X = df_validation.drop(columns = 'Y')
df_validation_Y = df_validation['Y']

df_test_X = df_test.drop(columns = 'Y')
df_test_Y = df_test['Y']

meta_model.fit(df_train_X, df_train_Y)
final_pred_train = meta_model.predict(df_train_X).clip(0,20)
final_pred_validation = meta_model.predict(df_validation_X).clip(0,20)
final_pred_test = meta_model.predict(df_test_X).clip(0,20)

print('Stacking | RMSE on Train Set:', np.sqrt(mean_squared_error(final_pred_train, df_train_Y)))
print('Stacking | RMSE on Validation Set:', np.sqrt(mean_squared_error(final_pred_validation, df_validation_Y)))
print(' XGBoost | RMSE on Validation Set:', ((df_validation.Y - df_validation.XGBoost)**2).mean()**0.5)
print('Catboost | RMSE on Validation Set:', ((df_validation.Y - df_validation.Catboost)**2).mean()**0.5)
print('LightGBM | RMSE on Validation Set:', ((df_validation.Y - df_validation.LightGBM)**2).mean()**0.5)

In [ ]:
# Saving Predictions
def submit_df(df_raw):
    df_out = df_raw.reset_index()
    df_out.columns = ['ID', 'item_cnt_month']
    return df_out

submit_df(pd.DataFrame(y_pred_test_xgb)).to_csv('y_pred_test_xgb.csv', index = False)
submit_df(pd.DataFrame(y_pred_test_cat)).to_csv('y_pred_test_cat.csv', index = False)
submit_df(pd.DataFrame(y_pred_test_light)).to_csv('y_pred_test_light.csv', index = False)
submit_df(pd.DataFrame(final_pred_test)).to_csv('y_pred_test_stack.csv', index = False)